# Assignment 2 - part-of-speech prediction from limited context

In this assignment, you will train classifiers that attempt, within a window of five words, to make a binary prediction about whether the third word belongs to a given part of speech (noun, verb, adjective, adverb), but using very limited information -- that is, the last two letters of the first, second, fourth, and fifth word of the sequence, and no information whatsoever directly from the third word itself.  You will strip out all punctuation (using the NLTK `WordPunctTokenizer`), lowercase, and remove stop words (using the NLTK English stop words list).

In other words, you will predict over samples that have two classes, P and not-P, where P is the selected part of speech to classify.  For example, from the sentence, "The quick brown fox jumped over the lazy dog.", we can select the following 5-word windows without stop words, "brown fox jumped lazy dog" and "quick brown fox jumped lazy".  If we select verbs as the part-of-speech we are classifying over, we get the instances <(wn,ox,zy,og),1>, since "jumped" is a verb, but <(ck,wn,ed,zy),0> because "fox" in that context is not.

This means that you will need to take into account the position of the last-two-letter feature:  "zy" as the fourth word's last two letters is different from "zy" as the fifth word's last two letters.  They are two features, say, `zy_4` and `zy_5`.

This will likely not actually work.  But it might!

You will create training and testing samples according to this procedure, and you will build a data structure that can be fed to a support vector machine (SVM) classifier.  You will train the classifier on the training data and evaluate it on the testing data. 

The work will be done in a .py module file in the same folder as this notebook.  **No modifications to this notebook will be graded.** We will run your module using this notebook or one we modify that you won't see in order to test your code.

The file you must create and add to the github repo is `mycode.py`, which will be imported here.  You can create your own notebooks or scripts to test it.  You can put any number of your own helper functions and also put optional parameters on any of the python functions mentioned here. You should also create a Markdown file, `notes.md`, to keep any **concise** notes and remarks about the assignment.  The code must run on mltgpu.

**This assignment is due Monday, 2022 March 7, at 23:59. There are 33 points and 5 bonus points.**

In [9]:
import mycode as mc

## Part 0 - preparation (2 points)

Fork this repository and create and add `mycode.py` and `notes.md`. 

## Part 1 - obtaining the text (3 points)

You will randomly select the given number of lines from the gzipped file we give you (so you will have to figure out how to access gzipped text files).  Explain how you implemented the random selection in `notes.md`. When we run it, it should give a new sample every time.

In [10]:
sampled_lines = mc.sample_lines("/scratch/UN-english.txt.gz", lines=100000)

print(len(sampled_lines))
sampled_lines[40000:40010]

100000


['The registration was set to begin at 8 a.m. but the actual time of starting varied from one registration point to another, with the latest starting time observed being 10.30.\n',
 'End-of-term examinations: 14 July-18 July\n',
 'Within the last year, the United Kingdom participated in the 20th IADC meeting, which was hosted by BNSC at Surrey University in April 2002.\n',
 'Source: Compiled from University statistics\n',
 'on Sustainable Development 12-13 July 2001\n',
 '4. By adopting such resolutions and instruments, the General Assembly has given clear expression to the overwhelming rejection by Member States of coercive measures and the strength of their opposition to the use of such measures against other States as a means of compulsion and of forcing them to accept policies that are not appropriate for or satisfactory to them.\n',
 'While preparations are under way to create national legislation in order to ratify the UN Convention against transnational organized crime, Austria 

## Part 2 - creating the samples (7 points)

From the sampled lines, you will then randomly create the five-word samples.

You will tokenize the sentences and apply POS-tagging to them -- you need to do this before you create the samples, since POS-tagging needs context. You will then remove stop words and punctuation and lowercase the remainder.  Next, you will randomly, over the entire set of sentences, choose samples of five words in sequence, up to a certain limit.  You find the last two characters of the first, second, fourth, and fifth words, and create the type of structure specified up in the introduction to this assignment for each sample. The exact representation is up to you.

In [11]:
processed_sentences = mc.process_sentences(sampled_lines)      
processed_sentences[40000:40010]

[[('expand', 'NNP'),
  ('existing', 'VBG'),
  ('framework', 'NN'),
  ('agreements', 'NNS'),
  ('promote', 'VB'),
  ('voluntary', 'JJ'),
  ('agreements', 'NNS'),
  ('sectoral', 'JJ'),
  ('regional', 'JJ'),
  ('international', 'JJ'),
  ('basis', 'NN'),
  ('decision', 'NN'),
  ('cop', 'NNP'),
  ('procedure', 'NN'),
  ('communication', 'NN'),
  ('information', 'NN'),
  ('september', 'NNP'),
  ('regional', 'NNP'),
  ('court', 'NNP'),
  ('berlin', 'NNP'),
  ('landgericht', 'NNP'),
  ('berlin', 'NNP'),
  ('convicted', 'VBD'),
  ('author', 'NN'),
  ('homicide', 'NN'),
  ('attempted', 'JJ'),
  ('homicide', 'NN'),
  ('several', 'JJ'),
  ('cases', 'NNS'),
  ('occurring', 'VBG'),
  ('sentencing', 'VBG'),
  ('prison', 'JJ'),
  ('term', 'NN'),
  ('six', 'CD'),
  ('years', 'NNS'),
  ('six', 'CD'),
  ('months', 'NNS'),
  ('may', 'NNP'),
  ('take', 'VBP'),
  ('assembly', 'NNP'),
  ('wishes', 'NNS'),
  ('consistent', 'JJ'),
  ('principles', 'NNS'),
  ('described', 'VBN'),
  ('paragraphs', 'NN'),
  ('pan

In [12]:
all_samples = mc.create_samples(processed_sentences, samples=50000)

print(all_samples[25000:25010])

[(('ng_1', 'ic_2', 'le_3', 'on_4'), 0), (('ng_1', 'en_2', 'na_3', 'es_4'), 0), (('on_1', 'ph_2', 've_3', 'ed_4'), 0), (('an_1', 'bt_2', 'ds_3', 'in_4'), 0), (('nt_1', 'le_2', 'on_3', 'ue_4'), 1), (('ry_1', 'ty_2', 'ed_3', 'le_4'), 0), (('ns_1', 'on_2', 'es_3', 'ul_4'), 0), (('ly_1', 'on_2', 'an_3', 'ed_4'), 0), (('us_1', 'ce_2', 'de_3', 'ld_4'), 0), (('an_1', 've_2', 'on_3', 've_4'), 0)]


## Part 3 - convert the samples into a Pandas DataFrame (10 points)

Here, you will take the samples and create a table whose columns are the features and the class and whose rows are the samples.  All the features and the class will be binary.  Note that there may be many columns, in the hundreds or thousands depending on the diversity of the final two consonants of the non-stop-words in the dataset, but the sum of all rows will be five.

In [13]:
fulldf = mc.create_df(all_samples)
fulldf[25000:25010]

,al_1,ve_2,en_3,on_4,eo_1,ed_2,ed_3,or_4,ic_1,on_2,...,uv_3,kr_2,mc_2,ić_1,án_3,uf_1,bh_3,wd_2,zl_3,v
25000,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
25005,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25006,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
25007,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
25008,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
25009,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Part 4 - extract training and testing sets (3 points)

Here, you will create the training and testing datasets in order to train the model.  This will be based on a test percentage.  Round up if the percentage does not divide evenly into the sample size.  You will need to separate the class column into the y-values for the classifier.

In [14]:
train_X, train_y, test_X, test_y = mc.split_samples(fulldf, test_percent=20)
len(train_X), len(train_y), len(test_X), len(test_y)

(40000, 40000, 10000, 10000)

## Part 5 - train models (3 points)

You will then train and return two support vector machine (SVM) models using the sklearn SVC class.  You should allow a choice between linear and radial basis function kernels.

In [15]:
model_linear = mc.train(train_X, train_y, kernel='linear')
model_rbf = mc.train(train_X, train_y, kernel='rbf')
model_linear, model_rbf

(SVC(kernel='linear'), SVC())

## Part 6 - evaluate the models (5 points)

You will calculate and print precision, recall, and F-measure for the models on the test data. In `notes.md`, write down your comparison of these simple measures on the two models and any thoughts you might have on what they mean. (It could be very short, and since the samples do not stay stable between runs, you can save the evaluation scores in `notes.md` too.)

In [16]:
mc.eval_model(model_linear, test_X, test_y)
mc.eval_model(model_rbf, test_X, test_y)

Modelname:  SVC(kernel='linear')
Precision:  0.14285714285714285 Recall:  0.0024691358024691358 F1:  0.004854368932038835
Modelname:  SVC()
Precision:  0.0 Recall:  0.0 F1:  0.0


/usr/local/lib64/python3.9/site-packages/sklearn/metrics/_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Part Bonus - try another sort of model from sklearn (5 points)

Write a separate, command-line script (not a notebook) uses `mycode.py` to do all of the above, except that it trains a non-SVM classifier model.  Any non-trivial model available in sklearn will do. Explain how to run your code and the results of your own evaluation in `notes.md`, including any observations or opinions you may have on the classifier method you used in comparison to SVM.

## Submission

Push to your fork of the GitHub repository (which must be made public) and submit the URL of your repository in Canvas.  You can submit this notebook with the output from your run, as long as you do not modify the code or text in it without permission from us.  